# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
num_years = 5

# when the point doesn't automatically geocode
# geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'
# lat_lon = [39.71824953362511, -104.96668328236497] # 'E 1st Ave and N Gilpin St, Denver, CO'

# geocoded_location = 'S Zuni St and W Arkansas Ave, Denver, CO'
# geocoded_location = 'Godsman Elementary School, Denver, CO'
# geocoded_location = 'W 26th Ave and Tennyson St, Denver, CO'
# geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'
# geocoded_location = 'E 32nd Ave and Sheridan Blvd, Denver, CO'
# geocoded_location = '700 E. Colfax Avenue, Denver, CO'
geocoded_location = 'Montview & Cherry, Denver, CO'

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.125

lat_lon = cda.geocode_location(geocoded_location)
lat_lon

[39.7474037, -104.933958]

In [4]:
query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int) #.to_clipboard()

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,5,0,0,0,0
2014,2,0,0,0,0
2015,4,0,0,0,0
2016,2,0,0,0,0
2017,5,0,0,0,0
2018,3,0,0,0,0
2019,9,0,1,0,1
2020,2,0,0,0,0
2021,1,0,0,0,0


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,41,0,1,0,1


In [13]:
display_df = nearby.copy()
display_df['bicycle'] = display_df['bicycle_ind'].replace(0, '').replace(1, True)
display_df['pedestrian'] = display_df['pedestrian_ind'].replace(0, '').replace(1, True)
display_df['serious_injury'] = display_df['sbi'].replace(False, '')
display_df['fatal'] = display_df['fatality'].replace(False, '')

display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle'
    , 'pedestrian'
    , 'serious_injury'
    , 'fatal'
    , 'distance_feet'
    , 'incident_id'
]

# nearby.to_clipboard()
# nearby[display_cols]
# display_df.loc[display_df.pedestrian_ind == 1, display_cols]
display_df[display_cols] #.tail(10)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle,pedestrian,serious_injury,fatal,distance_feet,incident_id
0,2013-06-08 20:19:00-06:00,TRAF - ACCIDENT,4600 BLOCK E MONTVIEW BLVD,,,,,201.786197,2013258723
1,2013-07-10 14:05:00-06:00,TRAF - ACCIDENT - HIT & RUN,2000 BLOCK N DEXTER ST,,,,,477.103412,2013318409
2,2013-07-10 20:09:00-06:00,TRAF - ACCIDENT - HIT & RUN,1900 BLOCK N CHERRY ST,,,,,439.894154,2013318407
3,2013-09-28 12:53:00-06:00,TRAF - ACCIDENT - HIT & RUN,2000 BLOCK N DEXTER ST,,,,,477.103412,2013467325
4,2013-11-02 16:28:00-06:00,TRAF - ACCIDENT - HIT & RUN,2000 BLOCK N DEXTER ST,,,,,477.103412,2013529255
5,2014-01-16 19:10:00-07:00,TRAF - ACCIDENT,E MONTVIEW BLVD / N CHERRY ST,,,,,2.779255,201427274
6,2014-05-10 15:29:00-06:00,TRAF - ACCIDENT - HIT & RUN,2000 BLOCK N DEXTER ST,,,,,477.103412,2014238051
7,2015-03-10 07:21:00-06:00,TRAF - ACCIDENT,N CHERRY ST / E MONTVIEW BLVD,,,,,2.779255,2015132268
8,2015-03-27 11:14:00-06:00,TRAF - ACCIDENT,2000 BLOCK N CHERRY ST,,,,,336.458715,2015166960
9,2015-04-28 12:51:00-06:00,TRAF - ACCIDENT,E MONTVIEW BLVD / N DAHLIA ST,,,,,533.299123,2015231872


In [8]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
folium.Circle(location=lat_lon, weight=2, radius=radius_miles*1609.34).add_to(this_map)
this_map.fit_bounds(
    [nearby[['geo_lat', 'geo_lon']].min().values.tolist()
     , nearby[['geo_lat', 'geo_lon']].max().values.tolist()]
)
this_map

In [9]:
# Text string with frequency of crashes

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "Montview & Cherry, Denver, CO" has had 9 traffic crashes within a 0.125 mile radius over the last 5 years, an average of 202.9 days between crashes. In that time, there were 0 deadly crashes and 0 crashes causing a serious bodily injury. There were 0 crashes involving a pedestrian and 0 crashes involving a bicyclist.


In [10]:
cda.conn.dispose()

## Bike and ped crashes in nearby dataframe

In [11]:
nearby_in_time_range[nearby_in_time_range.bicycle_ind > 0]

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,distance_feet,all_time


In [12]:
nearby_in_time_range[nearby_in_time_range.pedestrian_ind > 0]

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,distance_feet,all_time
